Anzeige der Temperaturdaten aus der Heizung

In [88]:
import sqlite3
import numpy as np
import pandas as pd
import os, sys
import ipywidgets as widgets
import matplotlib.pyplot as pl
DB_FILENAME = 'Temperaturen.db'


def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit("Stop right there!")
    
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        print('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        raise SystemExit("Stop right there!")
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
    

    
def plot_temp(start, dauer, dummy=False):

    data = df.tail(2000)
    #data.info()
    # UnixTime 	AussenTemp 	VorlaufTemp 	RuecklaufTemp 	WohnzimmerTemp
    data.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"])
    #data    



In [ ]:
# main
try:    
    df= get_df(DB_FILENAME)
except SystemExit as _e:
    print(_e)
    sys.exit(-1)
    
df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s')


dates_us = df.UnixTime.dt.strftime('%Y-%m-%d').unique()

df.set_index(['UnixTime'])
df['Aussen'] = str(df['AussenTemp'])
df['Wohnzimmer'] = str(df['WohnzimmerTemp'])

slider_dauer = widgets.IntSlider(start=1,end=100,step=1, description="Dauer")

widgets.interact(plot_temp, start= dates_us , dauer = slider_dauer)